In [1]:
import numpy as np
import pandas as pd
from quantile_norm import quantileNormalize
import datetime
import random

In [39]:
#hyperparameter
problem_set = 255116
# random sample students
student_cnt = 0
read_csv = False
limit_cnt = 300
data_file = str(problem_set)+'_sq_train_data.csv'
ps_file = str(problem_set)+'_ps_list'
ps = str(problem_set)+'_ps_index'

In [3]:
pl_df = pd.read_csv('this_one_problem_logs_seq.csv')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pl_df['formatted_start_time'] = pd.to_datetime(pl_df['start_time'])

date_before = datetime.date(2016, 8, 1)

print 'the number of students {}'.format(len(pl_df['user_id'].unique()))

pl_df = pl_df[pl_df['formatted_start_time'] < date_before].reset_index()

print 'the number of students {}'.format(len(pl_df['user_id'].unique()))

In [33]:
# read one experiment
ps_df = pd.read_csv('~/git/22-Experiments/'+str(problem_set)+'_exp.csv', header=None)

sublist = random.sample(pl_df['user_id'].unique(), student_cnt)
student_list = sublist + ps_df[1].unique().tolist()

train_df = pl_df[pl_df['user_id'].isin(student_list)].reset_index()

train_df = train_df[train_df['original'] == 1]

print 'number of row in train_df {}'.format(len(train_df))
print 'number of row in pl_df {}'.format(len(pl_df))

number of row in train_df 438676
number of row in pl_df 7744243


In [34]:
len(train_df['user_id'].unique())

545

In [35]:
print len(ps_df[1].unique())

545


In [40]:
if read_csv:
    ps_list = []
    with open(str(problem_set)+'_ps_list') as file:
        for line in file:
            line = line.strip()
            if line != '-':
                ps_list.append(int(line))
    mask = train_df['sequence_id'].isin(ps_list)
    ps_cnt = len(ps_list)
else:
    sub_counts = pd.value_counts(train_df['sequence_id'])
    ps_cnt = len(sub_counts[sub_counts > limit_cnt])
    mask = train_df['sequence_id'].isin(sub_counts[sub_counts > limit_cnt].index)
print ps_cnt

315


In [41]:
with open(ps_index, 'w') as f:
    for ite in sub_counts[sub_counts > limit_cnt].index.tolist():
        f.write(str(ite) + '\n')

In [19]:
ps_index = []
with open(str(problem_set)+'_ps_index', 'r') as f:
    for line in f:
        ps_index.append(int(line))

In [20]:
def prepare(sub_df, one_hot=True):
    new_df = pd.DataFrame()
    if one_hot:
        train_df['sequence_id'][~mask] = "-"
        sub_df = train_df
        new_df = pd.get_dummies(sub_df['sequence_id'])
    # u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time, first_action
    new_df['user_id'] = sub_df['user_id']
    # binary correctness
    new_df['correct'] = np.where(sub_df['correct'] < 1, 0, 1)
    new_df['bottom_hint'] = sub_df['bottom_hint']
    new_df['hint_count'] = sub_df['hint_count']
    new_df['attempt_count'] = sub_df['attempt_count']
    new_df['first_response_time'] = sub_df['first_response_time']
    # first action
    new_df['first_action'] = sub_df['first_action']

    #norm = lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() - x.min() else x/(x+1)
    # normalize within problems
    #new_df.insert(len(new_df.columns), 'NofFRT', sub_df.groupby('sequence_id')['first_response_time'].transform(norm))
    new_df.insert(len(new_df.columns), 'NofFRT', sub_df.groupby('sequence_id')['first_response_time'].rank(pct=True))
    del new_df['first_response_time']

    new_df.insert(len(new_df.columns), 'normalized_hint_count', sub_df.groupby('sequence_id')['hint_count'].rank(pct=True))
    del new_df['hint_count']

    new_df.insert(len(new_df.columns), 'normalized_attempt_count', sub_df.groupby('sequence_id')['attempt_count'].rank(pct=True))
    del new_df['attempt_count']
    del new_df['user_id']
    new_df['sequence_id'] = sub_df['sequence_id']
    new_df['user_id'] = sub_df['user_id']
    new_df['id'] = sub_df['id']
    return new_df

In [36]:
new_df = prepare(train_df, False)

In [22]:
def checknull(df):
    for column in df.columns:
        if df[column].isnull().values.any():
            print column

In [37]:
checknull(new_df)

In [38]:
new_df.to_csv(data_file, index=False)

In [ ]:
if not read_csv:
    with open(ps_file, 'w') as file:
        for ite in new_df.columns[:ps_cnt].tolist():
            file.write(str(ite) + "\n")

In [ ]:
pl_df = pd.read_csv(str(problem_set)+'_sq_train_data.csv')
# group by students
pl_df.set_index('id', inplace=True)
pl_g = pl_df.groupby('user_id')

In [ ]:
pl_df.head()

In [ ]:
for name, group in pl_g:
    for ite in group['sequence_id'].tolist():
        print ite
        break
    break

### stats info

In [ ]:
cnt_list = []
for name,group in sub_df.groupby('user_id'):
    cnt_list.append(len(group))

print np.mean(cnt_list)
print int(np.percentile(cnt_list, 75))

### Extra Features (Optional)

In [ ]:
start_complete_df = pd.read_csv('this_one_assignment_start_complete.csv')

In [ ]:
start_complete_df = start_complete_df.drop_duplicates()

In [ ]:
start_complete_df[start_complete_df['assignment_completed']==1].head()

In [ ]:
sub_df['assignment_id'].head()

In [ ]:
join_df = pd.merge(sub_df, start_complete_df, on=['assignment_id', 'user_id'], how='left')

In [ ]:
print len(join_df)
print len(sub_df)

In [ ]:
type_df = pd.read_csv('this_one_assignment_type.csv')

In [ ]:
type_df.head()

In [ ]:
type_df = type_df.drop_duplicates()

In [ ]:
join2_df = pd.merge(join_df, type_df, on=['assignment_id'], how='left')

In [ ]:
join2_df.head()

In [ ]:
print len(join2_df)

In [ ]:
pc_df = pd.read_csv('this_one_problem_correctness.csv')

In [ ]:
pc_df = pc_df.drop_duplicates()

In [ ]:
pc_df.head()

In [ ]:
join3_df = pd.merge(join2_df, pc_df, on=['problem_id'], how='left')

In [ ]:
join3_df.head()

In [ ]:
print len(join3_df)
print len(new_df)

In [ ]:
new_df.tail()

In [ ]:
join3_df.tail()

In [ ]:
del new_df['user_id']
del new_df['id']
new_df['assignment_completed'] = join3_df['assignment_completed']
new_df['is_flat_skill_builder'] = join3_df['is_flat_skill_builder']
new_df['textbook_items'] = join3_df['textbook_items']
new_df['certified_problem_solving_items'] = join3_df['certified_problem_solving_items']
new_df['correctness'] = join3_df['correctness']
new_df['user_id'] = join3_df['user_id']
new_df['id'] = join3_df['id']

In [ ]:
new_df['correctness'].fillna(0.5, inplace=True)

In [ ]:
new_df['assignment_completed'].fillna(0, inplace=True)
new_df['is_flat_skill_builder'].fillna(0, inplace=True)
new_df['textbook_items'].fillna(0, inplace=True)
new_df['certified_problem_solving_items'].fillna(0, inplace=True)

In [ ]:
new_df.dropna(inplace=True)

In [ ]:
for col in new_df.columns:
    if new_df[col].isnull().values.any():
        print col

In [ ]:
new_df.isnull().values.any()

In [ ]:
with open(str(problem_set)+'_ps_list', 'w') as file:
    for ite in new_df.columns[:ps_cnt].tolist():
        file.write(str(ite) + "\n")

In [ ]:
new_df.to_csv(str(problem_set)+'_extra_pl_train_data.csv', index=False)